In [ ]:
from dataset import import_dataset
from Oracle import Oracle
import pickle
import pandas as pd
import random
import networkx as nx
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import itertools

In [ ]:
from warnings import simplefilter
simplefilter(action='ignore', category=FutureWarning)

### Dataset

In [ ]:
# Dataset name
method = "SF"
#method = "CS"
#data_name = "OHSU"
#data_name = "Peking_1"
#data_name = "KKI"
#method = "contrast"
data_name = "autism"
#data_name = "bip80"
#data_name = "adhd_90"
#data_name = "adhdm_90"
#data_name = "bipolar90"

training = True


In [ ]:
# Import data
# Set the right folder !!!
data = import_dataset(data_name,folder="../data/")

In [ ]:
numeric_indices = [index for index in range(g.number_of_nodes())]
node_indices = sorted([node for node in g.nodes()])
assert numeric_indices == node_indices, "The node indexing is wrong."


### Classifier

In [ ]:
# Bipolar
oracle = Oracle(method,data,idc=2)

In [ ]:
oracle.train()

In [ ]:
g = data[0][0]
oracle.predict(g)

In [ ]:
# Evaluate
oracle.evaluate_classifier(data)

### Triangles-base

In [ ]:
def edit_distance(g_1,g_2):
    edg1 = set(g_1.edges())
    edg2 = set(g_2.edges())
    return round(len(edg1.symmetric_difference(edg2))/len(edg1.union(edg2)),3)

In [ ]:
def get_triangles(G, v):
    """Get all the triangles for a node
    Input:
        - Graph: G
        - node_id: v
    Output:
        - list: [(v_i,v_j)], where v_i and v_j are part of a triangle vit v.
    """
    neighbors1 = set(G.neighbors(v)) - {v}
    triangle_edges = []
    for nbr1, nbr2 in itertools.combinations(neighbors1, 2):
        if G.has_edge(nbr1, nbr2):
            triangle_edges.append((nbr1,nbr2))
    return triangle_edges

In [ ]:
def get_tri_sorting(g):
    """
    """
    d_tri = nx.triangles(g)
    e_empty = []
    e_fill = []
    for e1, e2 in itertools.combinations(g.nodes(), 2):
        e1_s = d_tri[e1]
        e2_s = d_tri[e2]
        if g.has_edge(e1,e2):
            e_empty.append((e1_s+e2_s,(e1,e2)))
        else:
            e_fill.append((e1_s+e2_s,(e1,e2)))
    e_empty.sort(reverse=True)
    e_fill.sort()
    return e_empty, e_fill

In [ ]:
def tri_based_search(g,e_empty,e_fill):
    """
    """
    y_hat = oracle.predict(g)
    y_c = y_hat
    i = 0
    g_c_i = g.copy()
    while((y_hat==y_c) & (i<(len(e_empty)-2))):
        #print(i,len(e_empty))
        e_e_i = e_empty[i][1]
        e_f_i = e_fill[i][1]
        # remove
        #print(e_e_i)
        g_c_i.remove_edge(e_e_i[0],e_e_i[1])
        y_c = oracle.predict(g_c_i)
        #print("i. remove: ",y_c)
        # add
        g_c_i.add_edge(e_f_i[0],e_f_i[1])
        y_c = oracle.predict(g_c_i)
        #print("i. add: ",y_c)
        i += 1
    return g_c_i,i-1

In [ ]:
%%time
info = []
for g,y in data:
    print(y,end="-")
    e_empty, e_fill = get_tri_sorting(g)
    g_c_i, iter_i = tri_based_search(g,e_empty,e_fill)
    ed_i = edit_distance(g,g_c_i)
    y_hat = oracle.predict(g)
    y_c = oracle.predict(g_c_i)
    #print(iter_i,ed_i)
    info.append((g,g_c_i,y,y_hat,y_c,iter_i,ed_i))

In [ ]:
len(info)

In [ ]:
#g,g_c_i,y_hat,y_c,iter_i,ed_i
c_cg_found = []
df_l = []
i = 0
for info_i in info:
    g, g_c_i, y, y_hat, y_c, iter_i, ed_i = info_i
    # is it a counterfactual?
    df_l.append([i,"All",y_hat,iter_i,ed_i])
    if y_hat!=y_c:
        v_c = True
        df_l.append([i,"C-found",y_hat,iter_i,ed_i])
        c_cg_found.append((i,g,g_c_i,y_hat,"TRI"))
    else:
        v_c = False
    # is it well-classified?
    if y_hat==y:
        c_ok = "Good"
    else:
        c_ok = "Miss"
    df_l.append([i,c_ok,y_hat,iter_i,ed_i])
    i += 1

In [ ]:
df = pd.DataFrame(df_l, columns=['g_id','Status','Predicted Class','Iter','Density'])

In [ ]:
COUNT_SEARCH = "tri"
df.to_pickle("results/"+"{}_{}_{}.pickle".format(data_name,method,COUNT_SEARCH))

In [ ]:
df = pd.DataFrame(c_cg_found, columns=['gid','G','C','y','Method'])
df.to_pickle("CG/"+"{}_{}_{}_cg.pickle".format(data_name,method,COUNT_SEARCH))

### dataset-based

In [ ]:
def edit_distance(g_1,g_2):
    edg1 = set(g_1.edges())
    edg2 = set(g_2.edges())
    return round(len(edg1.symmetric_difference(edg2))/len(edg1.union(edg2)),3)

In [ ]:
def dataset_strategy(gs,g_o,oracle,y_c):
    """
    """
    orac_call = 0
    candidate = []
    orac_call+=1
    for g_id,g_info in enumerate(gs):
        g_i,y = g_info
        y_cc = oracle.predict(g_i)
        orac_call+=1
        if y_cc==y_c:
            candidate.append((edit_distance(g_i,g_o),g_id))
    candidate.sort()
    ed_best, g_id_best = candidate[0]
    return ed_best, gs[g_id_best][0],orac_call,g_id_best

In [ ]:
info_dataset = []
ds_map = []
ds_results = []
i = 0
for g,y in data:
    print(i,end="-")
    y_hat = oracle.predict(g)
    y_c = 1-y_hat
    ds_edit_i,g_c_i,iter_count_i,g_c_id = dataset_strategy(data,g,oracle,y_c)
    ds_results.append(ds_edit_i)
    ds_map.append(g_c_id)
    i+=1
    info_dataset.append((g,g_c_i,y,y_hat,y_c,iter_count_i,ds_edit_i))

In [ ]:
#g,g_c_i,y_hat,y_c,iter_i,ed_i
c_cg_found = []
df_l = []
i = 0
for info_i in info_dataset:
    g, g_c_i, y, y_hat, y_c, iter_i, ed_i = info_i
    # is it a counterfactual?
    df_l.append([i,"All",y_hat,iter_i,ed_i])
    if y_hat!=y_c:
        v_c = True
        df_l.append([i,"C-found",y_hat,iter_i,ed_i])
        c_cg_found.append((i,g,g_c_i,y_hat,"DATASET"))
    else:
        v_c = False
    # is it well-classified?
    if y_hat==y:
        c_ok = "Good"
    else:
        c_ok = "Miss"
    df_l.append([i,c_ok,y_hat,iter_i,ed_i])
    i += 1

In [ ]:
df_dataset = pd.DataFrame(df_l, columns=['g_id','Status','Predicted Class','Iter','Density'])
COUNT_SEARCH = "dataset"
df_dataset.to_pickle("results/"+"{}_{}_{}.pickle".format(data_name,method,COUNT_SEARCH))

In [ ]:
df = pd.DataFrame(c_cg_found, columns=['gid','G','C','y','Method'])
COUNT_SEARCH = "dataset"
df.to_pickle("CG/"+"{}_{}_{}_cg.pickle".format(data_name,method,COUNT_SEARCH))

### Edge-based

In [ ]:
def edit_distance_raw(g_1,g_2):
    edg1 = set(g_1.edges())
    edg2 = set(g_2.edges())
    return len(edg1.symmetric_difference(edg2))
def get_change_list(g_1,g_2):
    edg1 = set(g_1.edges())
    edg2 = set(g_2.edges())
    return list(edg1.symmetric_difference(edg2))

In [ ]:
def bb(g,gc1,y_bar,k=5,l_max=2000):
    """
    """
    gc = gc1.copy()
    edges = get_change_list(g,gc)
    d = edit_distance_raw(g,gc)
    random.shuffle(edges)
    #
    li=0
    while(li<l_max and len(edges)>0 and d>1):
        ki = min(k,len(edges))
        gci = gc.copy()
        #get change list
        edges_i = [edges.pop(0) for i in range(ki)]
        #update
        for edge in edges_i:
            if gci.has_edge(*edge):
                gci.remove_edge(*edge)
                #print("Removed ",e)
            else:
                gci.add_edge(*edge)
                #print("Add ",e)
        #check
        r = oracle.predict(gci)
        li += 1
        if r==y_bar:
            gc = gci.copy()
            d = edit_distance_raw(g,gc)
            #print('ok --> ',r,d,l,k)
            #info.append((r,d,li,ki))
            k+=1
        else:
            d = edit_distance_raw(g,gc)
            #info.append((r,d,li,ki))
            if k>1:
                k-=1
                edges = edges + edges_i
    return gc,edit_distance(g,gc),li#,info

In [ ]:
%%time
info_edge = []
for g_id, g_info in enumerate(data):
    print(g_id,end="-")
    g,y = g_info
    g_c_i = data[ds_map[g_id]][0]
    #g_c_i = g.copy()
    y_hat = oracle.predict(g)
    gc,ed_i,iter_i = bb(g,g_c_i,1-y_hat,k=5,l_max=2000)
    y_c = oracle.predict(gc)
    info_edge.append((g,gc,y,y_hat,y_c,iter_i,ed_i))

In [ ]:
#g,g_c_i,y_hat,y_c,iter_i,ed_i
c_cg_found = []
df_l = []
i = 0
for info_i in info_edge:
    g, g_c_i, y, y_hat, y_c, iter_i, ed_i = info_i
    # is it a counterfactual?
    df_l.append([i,"All",y_hat,iter_i,ed_i])
    if y_hat!=y_c:
        v_c = True
        df_l.append([i,"C-found",y_hat,iter_i,ed_i])
        c_cg_found.append((i,g,g_c_i,y_hat,"DATASET+BW"))
    else:
        v_c = False
    # is it well-classified?
    if y_hat==y:
        c_ok = "Good"
    else:
        c_ok = "Miss"
    df_l.append([i,c_ok,y_hat,iter_i,ed_i])
    i += 1

In [ ]:
df_edge = pd.DataFrame(df_l, columns=['g_id','Status','Predicted Class','Iter','Density'])

In [ ]:
COUNT_SEARCH = "edge"
df_edge.to_pickle("results/"+"{}_{}_{}.pickle".format(data_name,method,COUNT_SEARCH))

In [ ]:
df = pd.DataFrame(c_cg_found, columns=['gid','G','C','y','Method'])
df.to_pickle("CG/"+"{}_{}_{}_cg.pickle".format(data_name,method,COUNT_SEARCH))

### Only Edge

In [ ]:
def edit_distance_raw(g_1,g_2):
    edg1 = set(g_1.edges())
    edg2 = set(g_2.edges())
    return len(edg1.symmetric_difference(edg2))

def obl_change(g_i):
    edg_r = list(set(g_i.edges()))
    cg = nx.complement(g_i)
    edg_a = list(set(cg.edges()))
    #
    #print(edg_a)
    random.shuffle(edg_r)
    random.shuffle(edg_a)
    return edg_r,edg_a

In [ ]:
def oblivious_search(g,y_bar,l_max=2000):
    """
    """
    edges_rem,edges_add = obl_change(g)
    #
    gci = g.copy()
    li=0
    while(li<l_max and len(edges_rem)>0):
        # select
        edges_r_i = edges_rem.pop(0)
        edges_a_i = edges_add.pop(0)
        # update
        #print(edges_r_i,edges_a_i)
        gci.remove_edge(*edges_r_i)
        gci.add_edge(*edges_a_i)
        # check
        r = oracle.predict(gci)
        li += 1
        if r==y_bar:
            return gci,edit_distance(g,gci),li#,info
    return g,0,l_max#,info

In [ ]:
%%time
info_edge = []
for g_id, g_info in enumerate(data):
    print(g_id,end="-")
    g,y = g_info
    y_hat = oracle.predict(g)
    gc,ed_i,iter_i = oblivious_search(g,1-y_hat,l_max=2000)
    y_c = oracle.predict(gc)
    info_edge.append((g,gc,y,y_hat,y_c,iter_i,ed_i))

In [ ]:
#g,g_c_i,y_hat,y_c,iter_i,ed_i
c_cg_found = []
df_l = []
i = 0
for info_i in info_edge:
    g, g_c_i, y, y_hat, y_c, iter_i, ed_i = info_i
    # is it a counterfactual?
    df_l.append([i,"All",y_hat,iter_i,ed_i])
    if y_hat!=y_c:
        v_c = True
        df_l.append([i,"C-found",y_hat,iter_i,ed_i])
        c_cg_found.append((i,g,g_c_i,y_hat,"EDG"))
    else:
        v_c = False
    # is it well-classified?
    if y_hat==y:
        c_ok = "Good"
    else:
        c_ok = "Miss"
    df_l.append([i,c_ok,y_hat,iter_i,ed_i])
    i += 1

In [ ]:
df_edge_obl = pd.DataFrame(df_l, columns=['g_id','Status','Predicted Class','Iter','Density'])

In [ ]:
COUNT_SEARCH = "edge obl"
df_edge_obl.to_pickle("results/"+"{}_{}_{}.pickle".format(data_name,method,COUNT_SEARCH))

In [ ]:
df = pd.DataFrame(c_cg_found, columns=['gid','G','C','y','Method'])
df.to_pickle("CG/"+"{}_{}_{}_cg.pickle".format(data_name,method,COUNT_SEARCH))